In [735]:
import pandas as pd
import numpy as np

In [736]:
CA_data = pd.read_csv('../CA_data.csv')

### Dropping useless column


In [737]:
# drop these columns
columns_dropped = ['Vehicle 1 Business Name', 'Lat1', 'Lat2', 'Lat3', 'Lng1', 'Lng2', 'Lng3', 
                   'Property Damage', 'Property Owner\'s Details', 'ZIPCODE of Accident', 
                   'State of Accident', 'County of Accident', 'Location of Accident', 'File ID'
                   ]
CA_data.drop(columns=columns_dropped, inplace=True)


In [738]:
print(len(CA_data.columns))

67


#### Column Weather Vehicle 1
We are grouping Fog/visiblity with Dark because they are visual impariements\
We are grouping slippery, wind with raining because when it rains, it usally reults in slipperiness and sometimes wind\

In [739]:
CA_data['Weather Vehicle 1'].replace(['Fog/Visibility', 'Dark'], 'Fog/Visibility/Dark', inplace=True)
CA_data['Weather Vehicle 1'].replace(['Slippery', 'Wind'], 'Raining', inplace=True)
CA_data.drop(CA_data[CA_data['Weather Vehicle 1'] == 'Not Available'].index, inplace=True)

print(CA_data['Weather Vehicle 1'].value_counts())

Clear                  265
Cloudy                  29
Raining                 14
Fog/Visibility/Dark      4
Name: Weather Vehicle 1, dtype: int64


In [740]:
# one hot encode
CA_data[['Clear', 'Cloudy','Fog/Visibility/Dark', 'Raining']] = pd.get_dummies(CA_data['Weather Vehicle 1'])
CA_data.drop(columns=['Weather Vehicle 1'], inplace=True)


#### Column: Vehicle 1 lighting
Grouping no unusla condition and daylight b/c they basically the same\
Grouping Dusk/dawn with dark with street lights b/c they are relatively similar conditions

also grouped dark with non functional street lights b/c too little instances of it


In [741]:
CA_data['Lighting Vehicle 1'].replace(['No unusual conditions'], 'Daylight', inplace=True)
CA_data['Lighting Vehicle 1'].replace(['Dusk/Dawn', 'Dark w Street-lights', 'Dark w Non-functioning Street-lights'], 'Dark', inplace=True)


print(CA_data['Lighting Vehicle 1'].value_counts())

Daylight    229
Dark         83
Name: Lighting Vehicle 1, dtype: int64


In [742]:
CA_data = pd.get_dummies(CA_data, columns = ['Lighting Vehicle 1'])


#### Column: Surface/Road condition 
Wet roads = slippery roads\
Dropped other and proceeding straight b/c they dont provide much value

In [743]:

CA_data['Roadway Surface Vehicle 1'].replace(['Slippery'], 'Wet', inplace=True)
CA_data.drop(CA_data[(CA_data['Roadway Surface Vehicle 1'] == 'Other') | 
                     (CA_data['Roadway Surface Vehicle 1'] == 'Proceeding Straight') |
                      (CA_data['Roadway Surface Vehicle 1'] == 'Not Available') ].index, inplace=True)


print(CA_data['Roadway Surface Vehicle 1'].value_counts())

Dry    286
Wet     22
Name: Roadway Surface Vehicle 1, dtype: int64


In [744]:
CA_data = pd.get_dummies(CA_data, columns = ['Roadway Surface Vehicle 1'], prefix = 'Road')


In [745]:
# if roadway conditions is no unsual condition, then it is 1 otherwise 0
CA_data['Roadway Conditions Vehicle 1'].replace(['No unusual conditions'], 0, inplace=True)
conditions = ['Construction - Repair Zone', 'Other', 'Construction on Roadway', 'Holes,Deep Rut', 'Reduced Roadway Width', 'Loose Material on Roadway']
CA_data['Roadway Conditions Vehicle 1'].replace(conditions, 1, inplace=True)
print(CA_data['Roadway Conditions Vehicle 1'].value_counts())


0.0    294
1.0     14
Name: Roadway Conditions Vehicle 1, dtype: int64


#### Column: Movement before the collision
grouping turns, high movements, parking, and dropping others

In [746]:
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Making Right Turn', 'Making Left Turn', 'Making U turn', 'Making Right Turn, Slowing/Stopping'], 'Turning', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Changing Lanes', 'Entering Traffic', 'Entrering Traffic','Xing into opposing lane', 'Passing Other Vehicle'], 'Moving on Highway', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Slowing/Stopping', 'Stopped in Traffic', 'Parked', 'Stopped, Merging'], 'Stopped', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Parking Manuerver', 'Parking Manuever'], 'Parking', inplace=True)
CA_data.drop(CA_data[(CA_data['Movement Preceding Collision Vehicle 1'] == 'Other')].index, inplace=True)

print(CA_data['Movement Preceding Collision Vehicle 1'].value_counts())

Stopped                151
Proceeding Straight     91
Turning                 29
Parking                 14
Moving on Highway       10
Backing                  8
Name: Movement Preceding Collision Vehicle 1, dtype: int64


In [747]:
CA_data = pd.get_dummies(CA_data, columns = ['Movement Preceding Collision Vehicle 1'])


#### Column: Vehicle 1 was stopped/moving in traffic
Fixing some off the mislabeled data


In [748]:

CA_data['Vehicle 1 was Stopped in Traffic'].replace(['/Off'], 'Yes', inplace=True)
CA_data['Vehicle 1 was Moving'].replace(['/Off'], 'No', inplace=True)

CA_data['Vehicle 1 was Stopped in Traffic'].replace(['yes'], 'Yes', inplace=True)
CA_data['Vehicle 1 was Moving'].replace(['Moving'], 'Yes', inplace=True)

CA_data.replace({'Vehicle 1 was Stopped in Traffic': {'Yes': 1, 'No': 0} ,'Vehicle 1 was Moving': {'Yes': 1, 'No': 0}}, inplace=True)


print(CA_data['Vehicle 1 was Stopped in Traffic'].value_counts())  
print(CA_data['Vehicle 1 was Moving'].value_counts())

0    192
1    165
Name: Vehicle 1 was Stopped in Traffic, dtype: int64
1    183
0    174
Name: Vehicle 1 was Moving, dtype: int64


#### Combining pedestrian, bicycle, and other entitys into one column called External_columns


In [749]:
print(CA_data['Pedestrian Involved in Accident (w V1)'].value_counts())
print(CA_data['Bicyclist involved in Accident (w V1)'].value_counts())
print(CA_data['Any other involved entity (w V1)'].value_counts())

No     356
Yes      1
Name: Pedestrian Involved in Accident (w V1), dtype: int64
No     349
Yes      8
Name: Bicyclist involved in Accident (w V1), dtype: int64
No                  328
Vehicle              13
Car                   5
Truck                 3
Parked                2
electric scooter      1
Scooter               1
IRON ROD              1
small dog             1
Van                   1
car                   1
Name: Any other involved entity (w V1), dtype: int64


In [750]:
def determine_new_col(row):
    pedestrian = row['Pedestrian Involved in Accident (w V1)'] == 'Yes'
    bicyclist = row['Bicyclist involved in Accident (w V1)'] == 'Yes'
    other = row['Any other involved entity (w V1)'].lower() not in ['no','vehicle','car','truck','van']
    if (pedestrian or bicyclist or other):
        return 1
    
    return 0

# Apply the function to each row
CA_data['External_factors'] = CA_data.apply(determine_new_col, axis=1)

print(CA_data['External_factors'].value_counts())

0    342
1     15
Name: External_factors, dtype: int64


In [751]:
# drop the other columns

columns_dropped = ['Pedestrian Involved in Accident (w V1)', 'Bicyclist involved in Accident (w V1)', 'Any other involved entity (w V1)']
CA_data.drop(columns=columns_dropped, inplace=True)

#### Column: Vehicle 1 Manufactuer

In [752]:
# rint(CA_data['Vehicle 1 Manufacturer'].value_counts())

# if waymo, cruise, google, apple, etc is in the name replace with the company name
def determine_vehicle_manufactuer(row):
    companies = ['waymo', 'cruise', 'google', 'apple', 'zoox']
    for company in companies:
        if company in row['Vehicle 1 Manufacturer'].lower():
            return company
    return "other"

CA_data['Vehicle 1 Manufacturer'] = CA_data.apply(determine_vehicle_manufactuer, axis=1)
print(CA_data['Vehicle 1 Manufacturer'].value_counts())



cruise    147
waymo     104
other      37
zoox       35
google     24
apple      10
Name: Vehicle 1 Manufacturer, dtype: int64


In [753]:
# 0ne hot encode
CA_data = pd.get_dummies(CA_data, columns = ['Vehicle 1 Manufacturer'], prefix = 'Manufacturer')



#### Column: Locational damage on the car

##### I am creating new columns that represent a general location where the car was damaged.


In [754]:
def rear_damage(row):
    if row['LR1'] == 'Yes' or row['RB'] == 'Yes' or row['RR1'] == 'Yes':
        return 1
    return 0

def front_damage(row):
    if row['LFC3'] == 'Yes' or row['RFC3'] == 'Yes' or row['FB'] == 'Yes':
        return 1
    return 0

def right_side_damage(row):
    right_side = ['RR3', 'RRP2', 'RRP4', 'FPS2', 'FPS4','RFC2']
    for side in right_side:
        if row[side] == 'Yes':
            return 1
    return 0

def left_side_damage(row):
    left_side = ['LR2', 'LRP1', 'LRP3', 'FDS1', 'FDS3', 'LFC1']
    for side in left_side:
        if row[side] == 'Yes':
            return 1
    return 0

def other_damage(row):
    other = ['LR3', 'LRP2', 'LRP4', 'FDS2', 'FDS4', 'LFC2', 
             'RR2','RRP1', 'RRP3', 'FPS1', 'FPS3', 'RFC1']
    for side in other:
        if row[side] == 'Yes':
            return 1
    return 0

In [755]:
CA_data['Rear Damage'] = CA_data.apply(rear_damage, axis=1)
CA_data['Front Damage'] = CA_data.apply(front_damage, axis=1)
CA_data['Right Side Damage'] = CA_data.apply(right_side_damage, axis=1)
CA_data['Left Side Damage'] = CA_data.apply(left_side_damage, axis=1)
CA_data['Inside Damage'] = CA_data.apply(other_damage, axis=1)

print(CA_data['Rear Damage'].value_counts())
print(CA_data['Front Damage'].value_counts())
print(CA_data['Right Side Damage'].value_counts())
print(CA_data['Left Side Damage'].value_counts())
print(CA_data['Inside Damage'].value_counts())



0    194
1    163
Name: Rear Damage, dtype: int64
0    309
1     48
Name: Front Damage, dtype: int64
0    304
1     53
Name: Right Side Damage, dtype: int64
0    291
1     66
Name: Left Side Damage, dtype: int64
0    343
1     14
Name: Inside Damage, dtype: int64


In [756]:
# Drop the columns that are not needed
columns_dropped = ['LR1', 'RB', 'RR1', 'LFC3', 'RFC3', 'FB', 
                   'RR3', 'RRP2', 'RRP4', 'FPS2', 'FPS4', 'RFC2', 
                   'LR2', 'LRP1', 'LRP3', 'FDS1', 'FDS3', 'LFC1', 
                   'LR3', 'LRP2', 'LRP4', 'FDS2', 'FDS4', 'LFC2', 
                   'RR2', 'RRP1', 'RRP3', 'FPS1', 'FPS3', 'RFC1']
CA_data.drop(columns=columns_dropped, inplace=True)

#### Column Date

##### Should i normalize?


In [757]:
CA_data["Date"] = pd.to_datetime(CA_data['Date'])
CA_data["accident_Year"] = CA_data["Date"].dt.year
CA_data['accident_Month'] = CA_data['Date'].dt.month
CA_data['accident_Day'] = CA_data['Date'].dt.day
CA_data['accident_Day_of_week'] = CA_data['Date'].dt.dayofweek
CA_data.drop(columns=['Date'], inplace=True)

#### Time

#### We only need the hour

In [758]:
CA_data.drop(CA_data[CA_data['Time'] == 'Not Available'].index, inplace=True)

CA_data['accident_Hour'] = pd.to_datetime(CA_data['Time']).dt.hour
CA_data.drop(columns = ['Time'], inplace=True)



#### Scaling some of the columns

In [759]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_scale = ['accident_Month', 'accident_Day', 'accident_Day_of_week', 'accident_Hour', 'accident_Year','Vehicle 1 Year of Manufacturing']
CA_data[columns_to_scale] = scaler.fit_transform(CA_data[columns_to_scale])


#### Column: Make and model

is nissan leaf a sedan or suv? sedan for now

In [760]:
# create a new column makeModel that combines the two columns
CA_data['makeModel'] = CA_data['Vehicle 1 Make'] + ' ' + CA_data['Vehicle 1 Model']

print(CA_data['makeModel'].value_counts())
# print(CA_data['Vehicle 1 Make'].value_counts())
# print(CA_data['Vehicle 1 Model'].value_counts())

top_values = CA_data['makeModel'].value_counts().index[:6]
encoded_data = pd.get_dummies(CA_data['makeModel'].where(CA_data['makeModel'].isin(top_values), other='other_makeModel'))
CA_data = pd.concat([CA_data, encoded_data], axis=1)



Chevrolet Bolt               104
Chrysler Pacifica             58
Jaguar I-Pace                 45
Cruise AV                     38
Toyota Highlander             32
Lexus RX 450h                 22
Lincoln MKZ                    7
Google Self Driving Car        5
Jaguar I-PACE                  4
Nissan Leaf                    4
Ford Fusion                    4
Toyota Prius                   2
NISSAN LEAF                    2
Cheverolet Bolt                2
Toyota Camry                   1
Hyundai KONA                   1
Lincoln MKZ Hybrid             1
Navya ARMA                     1
Volvo XC92                     1
Volvo XC91                     1
Volvo XC90                     1
Lexus LX 600H L                1
Hyundai Kona EV                1
Audi SQ5                       1
Mercedes-Benz S580             1
MERCEDES-BENZ S450 4MATIC      1
S CLASS MERCEDES-BENZ          1
Mercedes-Benz S500             1
MERCEDES-BENZ S580             1
MERCEDES-BENZ S-CLASS          1
Ford Fusio

In [761]:
# generated using chatgpt confirmed by gemini
suv = [
    'Jaguar I-Pace','Toyota Highlander', 'Lexus RX 450h', 'Jaguar I-PACE','Hyundai KONA','Volvo XC92',
    'Volvo XC91','Volvo XC90', 'Hyundai Kona EV','Audi SQ5'
]
sedans = ['Chevrolet Bolt', 'Cheverolet Bolt',  'Lincoln MKZ', 'Lincoln MKZ Hybrid', 
          'Ford Fusion', 'Ford Fusion Hybrid', 'Toyota Prius', 'Toyota Camry', 'Mercedes-Benz S580', 'MERCEDES-BENZ S450 4MATIC', 
          'Mercedes-Benz S500', 'MERCEDES-BENZ S580', 'MERCEDES-BENZ S-CLASS', 'Nissan Leaf', 'NISSAN LEAF', 'S CLASS MERCEDES-BENZ']

vans = ['Chrysler Pacifica', 'Lexus LX 600H L']
others = ['Cruise AV', 'Google Self Driving Car','Navya ARMA']

def determine_vehicle_type(row):
    
    if row['makeModel'] in suv:
        return 'SUV'
    elif row['makeModel'] in sedans:
        return 'Sedan'
    elif row['makeModel'] in vans:
        return 'Van'
    elif row['makeModel'] in others:
        return 'Other'
    return 'Forgot'

CA_data['Vehicle Type'] = CA_data.apply(determine_vehicle_type, axis=1)
print(CA_data['Vehicle Type'].value_counts())
CA_data = pd.get_dummies(CA_data, columns=['Vehicle Type'], prefix = 'Vehicle1_is')

Sedan    134
SUV      109
Van       59
Other     44
Name: Vehicle Type, dtype: int64


In [762]:
print(CA_data.columns)

Index(['Vehicle 1 Year of Manufacturing', 'Vehicle 1 Make', 'Vehicle 1 Model',
       'City of Accident', 'Vehicle 1 was Moving',
       'Vehicle 1 was Stopped in Traffic',
       'Number of Vehicles involved in Accident (w V1)', 'Vehicle Damage',
       'Vehicle 2 Model', 'Vehicle 2 Year of Manufacturing',
       'Vehicle 2 was Moving', 'Vehicle 2 was Stopped in Traffic',
       'Pedestrian Involved in Accident (w V2)',
       'Bicyclist involved in Accident (w V2)',
       'Any other involved entity (w V2)',
       'Number of Vehicles involved in Accident (w V2)', 'Car Mode',
       'Description', 'Weather Vehicle 2', 'Lighting Vehicle 2',
       'Roadway Surface Vehicle 2', 'Roadway Conditions Vehicle 1',
       'Roadway Conditions Vehicle 2',
       'Movement Preceding Collision Vehicle 2', 'Type of Collison Vehicle 1',
       'Type of Collison Vehicle 2', 'Other Factors', 'Clear', 'Cloudy',
       'Fog/Visibility/Dark', 'Raining', 'Lighting Vehicle 1_Dark',
       'Lighting Vehicl

In [763]:
CA_data.drop(columns=['makeModel', 'Vehicle 1 Make', 'Vehicle 1 Model'], inplace=True)


In [764]:
# save the csv
CA_data.to_csv('../CA_data_cleaned.csv', index=False)

#### Column City of accident

In [765]:
# print(CA_data['City of Accident'].value_counts())
bay_area = ['San Francisco', 'San Bruno', 'Crockett',  'Belmont', 'Emeryville']

# SF for some reason is not being replaced because
# for some reason San Fransico is being encoded weird or something
SF_var = CA_data['City of Accident'][:5][1]
print(SF_var, 'San Francisco', len(SF_var), len('San Francisco'))
print(f"Is the string and the var the same? {SF_var == 'San Francisco'}\n\n")

bay_area.append(SF_var)
south_bay = ['Mountain View', 'Palo Alto', 'Sunnyvale', 'Los Altos', 'Fremont',
                    'Santa Clara', 'Milpitas', 'San Jose', 'SANTA CLARA', 
                    'SB, 101 US Freeway, Old Middlefield Rd x Shorline Mountain View']
socal = ['Los Angeles', 'Santa Monica', 'LONG BEACH', 'GARDENA', 'SANTA FE SPRINGS', 'Irvine', ]

CA_data['City of Accident'].replace(south_bay, 'South Bay', inplace=True)
CA_data['City of Accident'].replace(socal, 'SoCal', inplace=True)
CA_data['City of Accident'].replace(bay_area , 'SF Bay Area', inplace=True)

CA_data.drop(CA_data[CA_data['City of Accident'] == 'Not Available'].index, inplace=True)

print(CA_data['City of Accident'].value_counts())

CA_data = pd.get_dummies(CA_data, columns=['City of Accident'], prefix = 'area_of_accident')


San Francisco San Francisco 13 13
Is the string and the var the same? False


SF Bay Area    234
South Bay       96
SoCal           14
Name: City of Accident, dtype: int64


#### Encode Vehicle Damage

In [766]:
# import label encoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
encoder = LabelEncoder()
print(CA_data['Vehicle Damage'].value_counts())

# drop the rows that have not available
CA_data.drop(CA_data[CA_data['Vehicle Damage'] == 'Not Available'].index, inplace=True)

# encode the columns
CA_data['Vehicle Damage'] = encoder.fit_transform(CA_data['Vehicle Damage'])
print(CA_data['Vehicle Damage'].value_counts())



Minor            217
None              49
Moderate          49
Not Available     21
Major              8
Name: Vehicle Damage, dtype: int64
1    217
3     49
2     49
0      8
Name: Vehicle Damage, dtype: int64


#### Car Mode

In [767]:
CA_data = pd.get_dummies(CA_data, columns = ['Car Mode'])

#### Description

In [768]:
CA_data.drop(columns=['Description'], inplace=True)

#### Type of collision

In [769]:
print(CA_data['Type of Collison Vehicle 1'].value_counts())
# drop not available
CA_data.drop(CA_data[CA_data['Type of Collison Vehicle 1'] == 'Not Available'].index, inplace=True)

CA_data = pd.get_dummies(CA_data, columns = ['Type of Collison Vehicle 1'], prefix = 'Collision')

Other            128
Rear End          72
Side Swipe        40
Head-On           22
Hit Object        20
Not Available     16
Broadside          5
Name: Type of Collison Vehicle 1, dtype: int64


#### Other factors

In [770]:
print(CA_data['Other Factors'].value_counts())
# Dropping it
CA_data.drop(columns=['Other Factors'], inplace=True)

None Apparent                                       251
CVC Sections Not Violated                            12
Entering/Leaving Traffic                             10
CVC SECTIONS Violated No                              3
Stop and Go Traffic                                   3
CVC Sections Violated                                 2
Runaway Vehicle                                       1
CVC SECTIONS Violated No, DEFECTIVE WEH EQUIP No      1
Other                                                 1
Vision Obscured                                       1
ENTERING/LEAVING RAMP                                 1
Inattention                                           1
Name: Other Factors, dtype: int64


#### Dealing with car 2

##### I am dropping all the descriptions of the second car and only indicating if there was another car

In [771]:
# columns to be dropped

# drop all the columns that have 'V2' oir 'Vehicle 2 in it
columns = [col for col in CA_data.columns if 'V2' in col or 'Vehicle 2' in col]
CA_data.drop(columns=columns, inplace=True)


In [772]:
print(CA_data.columns)


Index(['Vehicle 1 Year of Manufacturing', 'Vehicle 1 was Moving',
       'Vehicle 1 was Stopped in Traffic',
       'Number of Vehicles involved in Accident (w V1)', 'Vehicle Damage',
       'Roadway Conditions Vehicle 1', 'Clear', 'Cloudy',
       'Fog/Visibility/Dark', 'Raining', 'Lighting Vehicle 1_Dark',
       'Lighting Vehicle 1_Daylight', 'Road_Dry', 'Road_Wet',
       'Movement Preceding Collision Vehicle 1_Backing',
       'Movement Preceding Collision Vehicle 1_Moving on Highway',
       'Movement Preceding Collision Vehicle 1_Parking',
       'Movement Preceding Collision Vehicle 1_Proceeding Straight',
       'Movement Preceding Collision Vehicle 1_Stopped',
       'Movement Preceding Collision Vehicle 1_Turning', 'External_factors',
       'Manufacturer_apple', 'Manufacturer_cruise', 'Manufacturer_google',
       'Manufacturer_other', 'Manufacturer_waymo', 'Manufacturer_zoox',
       'Rear Damage', 'Front Damage', 'Right Side Damage', 'Left Side Damage',
       'Inside Dam

In [773]:
CA_data.to_csv('CA_data_cleaned.csv', index=False)